In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

In [15]:
dtype = torch.FloatTensor

sentences = [ "i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)  # 字典长度，number of Vocabulary

# NNLM Parameter
n_step = 2  # 窗口大小，n-1 in paper
n_hidden = 2  # h in paper
m = 2  # 最终词向量的长度，m in paper

In [19]:
word_dict

{'like': 0, 'milk': 1, 'dog': 2, 'i': 3, 'hate': 4, 'coffee': 5, 'love': 6}

In [20]:
number_dict

{0: 'like', 1: 'milk', 2: 'dog', 3: 'i', 4: 'hate', 5: 'coffee', 6: 'love'}

In [15]:
def make_batch(sentences):
    # 其实就是将句子对应成各自的向量，然后input_batch为前面的词，target_batch为要预测的词
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]
#         print("input, lbl：",input,target)

        input_batch.append(input)
        target_batch.append(target)

    return input_batch, target_batch

# Model
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, m) #投影矩阵
        self.H = nn.Parameter(torch.randn(n_step * m, n_hidden).type(dtype))  # 隐藏层的weight
        self.W = nn.Parameter(torch.randn(n_step * m, n_class).type(dtype))  # softmax层作用于词嵌入的weight
        self.d = nn.Parameter(torch.randn(n_hidden).type(dtype))  # 隐藏层的bias
        self.U = nn.Parameter(torch.randn(n_hidden, n_class).type(dtype))  # softmax层作用于hidden层的weight
        self.b = nn.Parameter(torch.randn(n_class).type(dtype))  # softmax层的bias

    def forward(self, X):
        X = self.C(X)  # 这里巧妙的采用embedding，取出第X行刚好就是该词我们想要的词嵌入
        X = X.view(-1, n_step * m)  # [数据个数即要训练样本数量, 窗口大小*词嵌入维度=每个训练样本的总的维度]
        tanh = torch.tanh(self.d + torch.mm(X, self.H))  # 全连接层 [batch_size, n_hidden]
        output = self.b + torch.mm(X, self.W) + torch.mm(tanh, self.U)  # softmax层[batch_size, n_class]
        return output

model = NNLM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch(sentences)
print(input_batch)
print(target_batch)
input_batch = torch.LongTensor(input_batch)  # embedded只能接受LongTensor
target_batch = torch.LongTensor(target_batch)

input, lbl： [3, 0] 2
input, lbl： [3, 6] 5
input, lbl： [3, 4] 1
[[3, 0], [3, 6], [3, 4]]
[2, 5, 1]


In [16]:
# Training
for epoch in range(5000):

    optimizer.zero_grad()
    output = model(input_batch)

    loss = criterion(output, target_batch)
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

Epoch: 1000 cost = 0.335694
Epoch: 2000 cost = 0.073772
Epoch: 3000 cost = 0.023977
Epoch: 4000 cost = 0.009994
Epoch: 5000 cost = 0.004726


In [17]:
# Predict
predict = model(input_batch).data.max(1, keepdim=True)[1] # .data只取出tensor中的数据，对于其他信息都丢弃

In [18]:
# Test
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
